# Agentic RAG Using CrewAI & LangChain

We are going to see how agents can be involved in the RAG system to rtrieve the most relevant information.

Source: 

Plaban Nayak, Build A Local Reliable RAG Agent Using CrewAI And Groq, https://medium.com/the-ai-forum/build-a-local-reliable-rag-agent-using-crewai-and-groq-013e5d557bcd

Notebook:

https://github.com/pavanbelagatti/Agentic-RAG-LangChain-CrewAI/blob/main/crew-agentic-pav.ipynb

In [1]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 sentence-transformers langchain-groq --quiet

In [2]:
import os
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from crewai_tools import PDFSearchTool

## Mention the Groq API Key

In [3]:
# Set the API key
GROQ_API_KEY = open("/Users/mjack6/.secrets/groq_mjack.apikey", "r").read().strip()
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

OPENAI_API_KEY = open("/Users/mjack6/.secrets/openai_mjack.apikey", "r").read().strip()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY 

## Mention the LLM being used

In [4]:
llm = ChatOpenAI(api_key=GROQ_API_KEY)

crew_llm = LLM(
    model="ollama/llama3-8b-8192",
    api_key=GROQ_API_KEY,
    max_tokens=1000,
    temperature=0.1
)

In [5]:
# Alternative Chat API:
# GEMINI_API_KEY = open("/Users/mjack6/.secrets/gemini_mjack.apikey", "r").read().strip()
# os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [6]:
# llm = ChatOpenAI(api_key=OPENAI_API_KEY)

# crew_llm = LLM(
#     model="gemini/gemini-1.5-flash",
#     api_key=GEMINI_API_KEY,
#     max_tokens=1000,
#     temperature=0.1
# )

In [7]:
import requests

pdf_url = 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'
response = requests.get(pdf_url)

with open('NIPS-2017-attention-is-all-you-need-Paper.pdf', 'wb') as file:
    file.write(response.content)

## Create a RAG tool variable to pass our PDF

In [8]:
rag_tool = PDFSearchTool(
    pdf='NIPS-2017-attention-is-all-you-need-Paper.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                # task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai_frameworks/lib/python3.11/site-packages/embedchain/embedder/huggingface.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=self.config.model, model_kwargs=self.config.model_kwargs)


In [9]:
# rag_tool = PDFSearchTool(
#     pdf='NIPS-2017-attention-is-all-you-need-Paper.pdf',
#     config=dict(
#         llm=dict(
#             provider="openai", # or google, openai, anthropic, llama2, ...
#             config=dict(
#                 model="llama3-8b-8192",
#                 # temperature=0.5,
#                 # top_p=1,
#                 # stream=true,
#             ),
#         ),
#         embedder=dict(
#             provider="openai", # or openai, ollama, ...
#             config=dict(
#                 model="llama3-8b-8192",
#                 # task_type="retrieval_document",
#                 # title="Embeddings",
#             ),
#         ),
#     )
# )

In [10]:
result = rag_tool.run("How did self-attention mechanism evolve in large language models?")

pprint(result)

Using Tool: Search a PDF's content
('Relevant Content:\n'
 'makes it more difﬁcult to learn dependencies between distant positions [ '
 '11]. In the Transformer this is reduced to a constant number of operations, '
 'albeit at the cost of reduced effective resolution due to averaging '
 'attention-weighted positions, an effect we counteract with Multi-Head '
 'Attention as described in section 3.2. Self-attention, sometimes called '
 'intra-attention is an attention mechanism relating different positions of a '
 'single sequence in order to compute a representation of the sequence. '
 'Self-attention has been used successfully in a variety of tasks including '
 'reading comprehension, abstractive summarization, textual entailment and '
 'learning task-independent sentence representations [4, 22, 23, 19]. '
 'End-to-end memory networks are based on a recurrent attention mechanism '
 'instead of sequence- aligned recurrence and have been shown to perform well '
 'on simple-language quest

## Mention the Tavily API Key

In [11]:
TAVILY_API_KEY = open("/Users/mjack6/.secrets/tavily_mjack.apikey", "r").read().strip()
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [12]:
web_search_tool = TavilySearchResults(k=3)

In [13]:
web_search_tool.run("What is self-attention mechansim in large language models?")

[{'url': 'https://medium.com/data-science-community-srm/exploring-attention-mechanisms-in-large-language-models-33549ae2c5b8',
  'content': 'Imagine each word in a sentence as a friend in a group chat. The self-attention mechanism lets the model look at each friend’s message, and based on what they said, decide how much attention to give to each one. This helps the model figure out the context and meaning of the words, making them really good at understanding language. [...] In the Transformer architecture, the self-attention mechanism plays a crucial role by helping the model decide how important each word or token is in a sequence. It does this by giving attention scores to words based on how relevant they are to others in the sequence. This way, the model can focus on the most contextually important information. [...] Now, self-attention is a feature in their design that helps them understand words and language better. It’s like when you read a sentence, and you give more attention 

## Define a Tool

In [14]:
@tool
def router_tool(question):
  """Router Function"""
  if 'self-attention' in question:
    return 'vectorstore'
  else:
    return 'web_search'

## Create Agents to work with

In [15]:
Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [16]:
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [17]:
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [18]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [19]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

## Defining Tasks

In [20]:
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [21]:
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)

In [22]:
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [23]:
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

## Define a flow for the use case

In [24]:
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)

## Ask any query

In [25]:
inputs = {"question":"How does self-attention mechanism help large language models?"}

## Kick off the agent pipeline

In [26]:
result = rag_crew.kickoff(inputs=inputs)

# Agent: Router
## Task: Analyse the keywords in the question How does self-attention mechanism help large language models?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.
 

I encountered an error while trying to use the tool. This was the error: router_tool() missing 1 required positional argument: 'question'.
 Tool router_tool accepts these inputs: Tool Name: router_tool
Tool Arguments: {}
Tool Description: Router Function



# Agent: Router
## Using tool: router_tool
## Tool Input: 
"{}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: router_tool() missing 1 required positional argument: 'question'.
 Tool router_tool accepts these inputs: Tool Name: router_tool
Tool Arguments: {}
Tool Description: Router Fu

## Obtain the final response

In [27]:
pprint(result.raw)

('Based on the previous response from the hallucination task indicating that '
 'the answer is useful, I can confirm that the self-attention mechanism helps '
 'large language models by allowing the models to weigh the significance of '
 'different words in a sequence, enabling more effective learning and '
 'contextual understanding.')
